In [1]:
from allennlp_models.pretrained import load_predictor
from allennlp_models.pretrained import get_pretrained_models

import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

from checklist.pred_wrapper import PredictorWrapper
import pandas as pd

from nltk.corpus import wordnet as wn
import random
pd.set_option('max_colwidth', 1000)

In [6]:
# you can define the amount of samples here!
nsamples = 1

In [3]:
def get_tags_v0(sentence):
    output = srl_predictor.predict(sentence)
    if len(output['verbs']) >= 1:
        pp_tags = output['verbs'][0]['tags']#['verb']#[0]['tags']
    else:
        pp_tags = []
    return pp_tags

def get_descr_v0(sentence):
    output = srl_predictor.predict(sentence)
    if len(output['verbs']) >= 1:
        pp_descr = output['verbs'][0]['description']
    else:
        pp_descr = []
    return pp_descr

def find_final_min_one_arg(list_of_arg):
    if list_of_arg:
        element = list_of_arg[-2]
    else:
        element = 'missed_verb'
    return element

def find_first_arg(list_of_arg):
    if list_of_arg:
        element = list_of_arg[0]
    else:
        element = 'missed_verb'
    return element

def inverse_topic(sentence):
    string_split = sentence.split()
    loc = string_split[-3:]
    loc[0]= loc[0].capitalize()
    loc[2] = loc[2][:-1]
    loc[2] = loc[2]+','
    not_loc = string_split[:-3]
    not_loc[0] = not_loc[0].lower()
    rebuilt_sentence_list = loc + not_loc
    rebuilt_sentence = " ".join(rebuilt_sentence_list)+"."
    return(rebuilt_sentence)

def make_data_locative(nsamples):
    list_locative_normal= []
    list_locative_inverted = []
    editor = Editor()

    list_of_places_to_sleep = ['bed', 'hut', 'park', 'tent','cage']

    test_data = editor.template("Only {first_name} sleeps in that {place}.",
                                place = list_of_places_to_sleep,
                                meta=True,  
                                nsamples=nsamples,
                                remove_duplicates=True)
    list_locative_normal.append(test_data['data'])
    #flatten list
    list_locative_normal = [x for xs in list_locative_normal for x in xs]
    
    return list_locative_normal

def make_data_directive(nsamples):
    editor = Editor()
    list_directive = []

    places = ['classroom', 'party', 'park', 'tent','cage','building','future','past','wedding','funeral']
    verb = ['went to']
    subj = ['girls', 'boys', 'men', 'women', 'animals', 'horses']

    test_data = editor.template("The {subj} {verb} the {place}.",
                                subj = subj,
                                place = places,
                                verb = verb,
                                meta=True,  
                                nsamples=nsamples,
                                remove_duplicates=True)
    list_directive.append(test_data['data'])
    #flatten list
    list_directive = [x for xs in list_directive for x in xs]
    return list_directive

## In the cell below, the functions are applied to the data (normal / inverted). All output is stored in separate columns. After that, all of the rows where the tags for normal / inverted differ are shown and the failure rate is calculated by dividing the wrongly predicted tag by the lenght of the original dataframe 

In [4]:
list_data_locative = make_data_locative(nsamples)
list_data_directive = make_data_directive(nsamples)

In [8]:
srl_predictor = load_predictor('structured-prediction-srl-bert')
df_locative = pd.DataFrame(list_data_locative)

df_locative['tags_normal'] = df_locative[0].apply(get_tags_v0)
df_locative['descr_normal'] = df_locative[0].apply(get_descr_v0)
df_locative['final-1_tag_normal'] = df_locative['tags_normal'].apply(find_final_min_one_arg)

df_locative['inverted'] = df_locative[0].apply(inverse_topic)
df_locative['tags_inverted'] = df_locative['inverted'].apply(get_tags_v0)
df_locative['descr_inverted'] = df_locative['inverted'].apply(get_descr_v0)
df_locative['first_tag_inverted'] = df_locative['tags_inverted'].apply(find_first_arg)

df_locative['final-1_tag_normal'] = df_locative['final-1_tag_normal'].str[2:]
df_locative['first_tag_inverted'] =df_locative['first_tag_inverted'].str[2:]
display(df_locative.head(5))

df_locative[0].to_csv("../datasets/inversion/locative.txt", header=None, index=False)
df_locative.to_csv("../results/inversion/locative_results_bert.csv")

lerc is not a registered model.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,0,tags_normal,descr_normal,final-1_tag_normal,inverted,tags_inverted,descr_inverted,first_tag_inverted
0,Only Leslie sleeps in that park.,"[B-ARG0, I-ARG0, B-V, B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O]",[ARG0: Only Leslie] [V: sleeps] [ARGM-LOC: in that park] .,ARGM-LOC,"In that park, only Leslie sleeps.","[B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O, B-ARG0, I-ARG0, B-V, O]","[ARGM-LOC: In that park] , [ARG0: only Leslie] [V: sleeps] .",ARGM-LOC
1,Only Ben sleeps in that bed.,"[B-ARG0, I-ARG0, B-V, B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O]",[ARG0: Only Ben] [V: sleeps] [ARGM-LOC: in that bed] .,ARGM-LOC,"In that bed, only Ben sleeps.","[B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O, B-ARG0, I-ARG0, B-V, O]","[ARGM-LOC: In that bed] , [ARG0: only Ben] [V: sleeps] .",ARGM-LOC
2,Only Lucy sleeps in that bed.,"[B-ARG0, I-ARG0, B-V, B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O]",[ARG0: Only Lucy] [V: sleeps] [ARGM-LOC: in that bed] .,ARGM-LOC,"In that bed, only Lucy sleeps.","[B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O, B-ARG0, I-ARG0, B-V, O]","[ARGM-LOC: In that bed] , [ARG0: only Lucy] [V: sleeps] .",ARGM-LOC
3,Only Jim sleeps in that tent.,"[B-ARG0, I-ARG0, B-V, B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O]",[ARG0: Only Jim] [V: sleeps] [ARGM-LOC: in that tent] .,ARGM-LOC,"In that tent, only Jim sleeps.","[B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O, B-ARG0, I-ARG0, B-V, O]","[ARGM-LOC: In that tent] , [ARG0: only Jim] [V: sleeps] .",ARGM-LOC
4,Only Joseph sleeps in that hut.,"[B-ARG0, I-ARG0, B-V, B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O]",[ARG0: Only Joseph] [V: sleeps] [ARGM-LOC: in that hut] .,ARGM-LOC,"In that hut, only Joseph sleeps.","[B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O, B-ARG0, I-ARG0, B-V, O]","[ARGM-LOC: In that hut] , [ARG0: only Joseph] [V: sleeps] .",ARGM-LOC


In [9]:
errors_locative_bert = df_locative[df_locative['final-1_tag_normal'] != df_locative['first_tag_inverted']][[0,'descr_normal', 'descr_inverted']]
failure_rate_locative_bert = len(errors_locative_bert) / len(df_locative)
errors_locative_bert

,0,descr_normal,descr_inverted
81,Only Louis sleeps in that tent.,[ARG0: Only Louis] [V: sleeps] [ARGM-LOC: in that tent] .,[]
84,Only Harriet sleeps in that hut.,[ARG0: Only Harriet] [V: sleeps] [ARGM-LOC: in that hut] .,[]
93,Only Dorothy sleeps in that bed.,[ARG0: Only Dorothy] [V: sleeps] [ARGM-LOC: in that bed] .,[]
94,Only Ruth sleeps in that cage.,[ARG0: Only Ruth] [V: sleeps] [ARGM-LOC: in that cage] .,[]


In [10]:
df_directive = pd.DataFrame(list_data_directive)

df_directive['tags_normal'] = df_directive[0].apply(get_tags_v0)
df_directive['descr_normal'] = df_directive[0].apply(get_descr_v0)
df_directive['final-1_tag_normal'] = df_directive['tags_normal'].apply(find_final_min_one_arg)

df_directive['inverted'] = df_directive[0].apply(inverse_topic)
df_directive['tags_inverted'] = df_directive['inverted'].apply(get_tags_v0)
df_directive['descr_inverted'] = df_directive['inverted'].apply(get_descr_v0)
df_directive['first_tag_inverted'] = df_directive['tags_inverted'].apply(find_first_arg)

df_directive['final-1_tag_normal'] = df_directive['final-1_tag_normal'].str[2:]
df_directive['first_tag_inverted'] =df_directive['first_tag_inverted'].str[2:]
display(df_directive.head(5))

df_directive[0].to_csv("../datasets/inversion/directive.txt", header=None, index=False)
df_directive.to_csv("../results/inversion/directive_results_bert.csv")

,0,tags_normal,descr_normal,final-1_tag_normal,inverted,tags_inverted,descr_inverted,first_tag_inverted
0,The men went to the future.,"[B-ARG0, I-ARG0, B-V, B-ARG4, I-ARG4, I-ARG4, O]",[ARG0: The men] [V: went] [ARG4: to the future] .,ARG4,"To the future, the men went.","[B-ARG4, I-ARG4, I-ARG4, O, B-ARG0, I-ARG0, B-V, O]","[ARG4: To the future] , [ARG0: the men] [V: went] .",ARG4
1,The men went to the party.,"[B-ARG0, I-ARG0, B-V, B-ARG4, I-ARG4, I-ARG4, O]",[ARG0: The men] [V: went] [ARG4: to the party] .,ARG4,"To the party, the men went.","[B-ARG4, I-ARG4, I-ARG4, O, B-ARG0, I-ARG0, B-V, O]","[ARG4: To the party] , [ARG0: the men] [V: went] .",ARG4
2,The animals went to the park.,"[B-ARG0, I-ARG0, B-V, B-ARG4, I-ARG4, I-ARG4, O]",[ARG0: The animals] [V: went] [ARG4: to the park] .,ARG4,"To the park, the animals went.","[B-ARG4, I-ARG4, I-ARG4, O, B-ARG0, I-ARG0, B-V, O]","[ARG4: To the park] , [ARG0: the animals] [V: went] .",ARG4
3,The women went to the cage.,"[B-ARG0, I-ARG0, B-V, B-ARG4, I-ARG4, I-ARG4, O]",[ARG0: The women] [V: went] [ARG4: to the cage] .,ARG4,"To the cage, the women went.","[B-ARG4, I-ARG4, I-ARG4, O, B-ARG0, I-ARG0, B-V, O]","[ARG4: To the cage] , [ARG0: the women] [V: went] .",ARG4
4,The animals went to the wedding.,"[B-ARG0, I-ARG0, B-V, B-ARG4, I-ARG4, I-ARG4, O]",[ARG0: The animals] [V: went] [ARG4: to the wedding] .,ARG4,"To the wedding, the animals went.","[B-ARG4, I-ARG4, I-ARG4, O, B-ARG0, I-ARG0, B-V, O]","[ARG4: To the wedding] , [ARG0: the animals] [V: went] .",ARG4


In [11]:
errors_directive_bert = df_directive[df_directive['final-1_tag_normal'] != df_directive['first_tag_inverted']][[0,'descr_normal', 'descr_inverted']]
failure_rate_directive_bert = len(errors_directive_bert) / len(df_directive)
errors_directive_bert


,0,descr_normal,descr_inverted


In [12]:
srl_predictor = load_predictor('structured-prediction-srl')
df_locative = pd.DataFrame(list_data_locative)

df_locative['tags_normal'] = df_locative[0].apply(get_tags_v0)
df_locative['descr_normal'] = df_locative[0].apply(get_descr_v0)
df_locative['final-1_tag_normal'] = df_locative['tags_normal'].apply(find_final_min_one_arg)

df_locative['inverted'] = df_locative[0].apply(inverse_topic)
df_locative['tags_inverted'] = df_locative['inverted'].apply(get_tags_v0)
df_locative['descr_inverted'] = df_locative['inverted'].apply(get_descr_v0)
df_locative['first_tag_inverted'] = df_locative['tags_inverted'].apply(find_first_arg)

df_locative['final-1_tag_normal'] = df_locative['final-1_tag_normal'].str[2:]
df_locative['first_tag_inverted'] =df_locative['first_tag_inverted'].str[2:]
df_locative.head(5)

lerc is not a registered model.


,0,tags_normal,descr_normal,final-1_tag_normal,inverted,tags_inverted,descr_inverted,first_tag_inverted
0,Only Leslie sleeps in that park.,"[B-ARG0, I-ARG0, B-V, B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O]",[ARG0: Only Leslie] [V: sleeps] [ARGM-LOC: in that park] .,ARGM-LOC,"In that park, only Leslie sleeps.","[B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O, B-ARG0, I-ARG0, B-V, O]","[ARGM-LOC: In that park] , [ARG0: only Leslie] [V: sleeps] .",ARGM-LOC
1,Only Ben sleeps in that bed.,"[B-ARG0, I-ARG0, B-V, B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O]",[ARG0: Only Ben] [V: sleeps] [ARGM-LOC: in that bed] .,ARGM-LOC,"In that bed, only Ben sleeps.","[B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O, B-ARG0, I-ARG0, B-V, O]","[ARGM-LOC: In that bed] , [ARG0: only Ben] [V: sleeps] .",ARGM-LOC
2,Only Lucy sleeps in that bed.,"[B-ARG0, I-ARG0, B-V, B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O]",[ARG0: Only Lucy] [V: sleeps] [ARGM-LOC: in that bed] .,ARGM-LOC,"In that bed, only Lucy sleeps.","[B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O, B-ARG0, I-ARG0, B-V, O]","[ARGM-LOC: In that bed] , [ARG0: only Lucy] [V: sleeps] .",ARGM-LOC
3,Only Jim sleeps in that tent.,"[B-ARG0, I-ARG0, B-V, B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O]",[ARG0: Only Jim] [V: sleeps] [ARGM-LOC: in that tent] .,ARGM-LOC,"In that tent, only Jim sleeps.","[B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O, B-ARG0, I-ARG0, B-V, O]","[ARGM-LOC: In that tent] , [ARG0: only Jim] [V: sleeps] .",ARGM-LOC
4,Only Joseph sleeps in that hut.,"[B-ARG0, I-ARG0, B-V, B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O]",[ARG0: Only Joseph] [V: sleeps] [ARGM-LOC: in that hut] .,ARGM-LOC,"In that hut, only Joseph sleeps.","[B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O, B-ARG0, I-ARG0, B-V, O]","[ARGM-LOC: In that hut] , [ARG0: only Joseph] [V: sleeps] .",ARGM-LOC


In [13]:
errors_locative_bilstm = df_locative[df_locative['final-1_tag_normal'] != df_locative['first_tag_inverted']][[0,'descr_normal', 'descr_inverted']]
failure_rate_locative_bilstm = len(errors_locative_bilstm) / len(df_locative)
errors_locative_bilstm

,0,descr_normal,descr_inverted
66,Only Rebecca sleeps in that hut.,[ARG0: Only Rebecca] [V: sleeps] [ARGM-MNR: in that hut] .,"[ARGM-LOC: In that hut] , [ARG0: only Rebecca] [V: sleeps] ."
81,Only Louis sleeps in that tent.,[ARG0: Only Louis] [V: sleeps] [ARGM-LOC: in that tent] .,[]
84,Only Harriet sleeps in that hut.,[ARG0: Only Harriet] [V: sleeps] [ARGM-LOC: in that hut] .,[]
93,Only Dorothy sleeps in that bed.,[ARG0: Only Dorothy] [V: sleeps] [ARGM-LOC: in that bed] .,[]
94,Only Ruth sleeps in that cage.,[ARG0: Only Ruth] [V: sleeps] [ARGM-LOC: in that cage] .,[]


In [14]:
df_locative.iloc[27]

0                                                       Only Dick sleeps in that bed.
tags_normal              [B-ARG0, I-ARG0, B-V, B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O]
descr_normal                  [ARG0: Only Dick] [V: sleeps] [ARGM-LOC: in that bed] .
final-1_tag_normal                                                           ARGM-LOC
inverted                                               In that bed, only Dick sleeps.
tags_inverted         [B-ARGM-LOC, I-ARGM-LOC, I-ARGM-LOC, O, B-ARG0, I-ARG0, B-V, O]
descr_inverted              [ARGM-LOC: In that bed] , [ARG0: only Dick] [V: sleeps] .
first_tag_inverted                                                           ARGM-LOC
Name: 27, dtype: object

In [15]:
df_directive = pd.DataFrame(list_data_directive)

df_directive['tags_normal'] = df_directive[0].apply(get_tags_v0)
df_directive['descr_normal'] = df_directive[0].apply(get_descr_v0)
df_directive['final-1_tag_normal'] = df_directive['tags_normal'].apply(find_final_min_one_arg)

df_directive['inverted'] = df_directive[0].apply(inverse_topic)
df_directive['tags_inverted'] = df_directive['inverted'].apply(get_tags_v0)
df_directive['descr_inverted'] = df_directive['inverted'].apply(get_descr_v0)
df_directive['first_tag_inverted'] = df_directive['tags_inverted'].apply(find_first_arg)

df_directive['final-1_tag_normal'] = df_directive['final-1_tag_normal'].str[2:]
df_directive['first_tag_inverted'] =df_directive['first_tag_inverted'].str[2:]
df_directive.head(5)

,0,tags_normal,descr_normal,final-1_tag_normal,inverted,tags_inverted,descr_inverted,first_tag_inverted
0,The men went to the future.,"[B-ARG0, I-ARG0, B-V, B-ARG4, I-ARG4, I-ARG4, O]",[ARG0: The men] [V: went] [ARG4: to the future] .,ARG4,"To the future, the men went.","[B-ARGM-TMP, I-ARGM-TMP, I-ARGM-TMP, O, B-ARG0, I-ARG0, B-V, O]","[ARGM-TMP: To the future] , [ARG0: the men] [V: went] .",ARGM-TMP
1,The men went to the party.,"[B-ARG0, I-ARG0, B-V, B-ARG4, I-ARG4, I-ARG4, O]",[ARG0: The men] [V: went] [ARG4: to the party] .,ARG4,"To the party, the men went.","[B-ARG2, I-ARG2, I-ARG2, O, B-ARG0, I-ARG0, B-V, O]","[ARG2: To the party] , [ARG0: the men] [V: went] .",ARG2
2,The animals went to the park.,"[B-ARG1, I-ARG1, B-V, B-ARG4, I-ARG4, I-ARG4, O]",[ARG1: The animals] [V: went] [ARG4: to the park] .,ARG4,"To the park, the animals went.","[B-ARG4, I-ARG4, I-ARG4, O, B-ARG1, I-ARG1, B-V, O]","[ARG4: To the park] , [ARG1: the animals] [V: went] .",ARG4
3,The women went to the cage.,"[B-ARG0, I-ARG0, B-V, B-ARG4, I-ARG4, I-ARG4, O]",[ARG0: The women] [V: went] [ARG4: to the cage] .,ARG4,"To the cage, the women went.","[B-ARG4, I-ARG4, I-ARG4, O, B-ARG0, I-ARG0, B-V, O]","[ARG4: To the cage] , [ARG0: the women] [V: went] .",ARG4
4,The animals went to the wedding.,"[B-ARG1, I-ARG1, B-V, B-ARG4, I-ARG4, I-ARG4, O]",[ARG1: The animals] [V: went] [ARG4: to the wedding] .,ARG4,"To the wedding, the animals went.","[B-ARG4, I-ARG4, I-ARG4, O, B-ARG1, I-ARG1, B-V, O]","[ARG4: To the wedding] , [ARG1: the animals] [V: went] .",ARG4


In [16]:
errors_directive_bilstm = df_directive[df_directive['final-1_tag_normal'] != df_directive['first_tag_inverted']][[0,'descr_normal', 'descr_inverted']]
failure_rate_directive_bilstm = len(errors_directive_bilstm) / len(df_directive)
errors_directive_bilstm

,0,descr_normal,descr_inverted
0,The men went to the future.,[ARG0: The men] [V: went] [ARG4: to the future] .,"[ARGM-TMP: To the future] , [ARG0: the men] [V: went] ."
1,The men went to the party.,[ARG0: The men] [V: went] [ARG4: to the party] .,"[ARG2: To the party] , [ARG0: the men] [V: went] ."
6,The animals went to the party.,[ARG1: The animals] [V: went] [ARG4: to the party] .,"[ARG2: To the party] , [ARG1: the animals] [V: went] ."
8,The girls went to the party.,[ARG0: The girls] [V: went] [ARG4: to the party] .,"[ARG2: To the party] , [ARG0: the girls] [V: went] ."
19,The horses went to the future.,[ARG1: The horses] [V: went] [ARG4: to the future] .,"[ARGM-TMP: To] [ARG4: the future] , [ARG1: the horses] [V: went] ."
20,The boys went to the future.,[ARG0: The boys] [V: went] [ARG4: to the future] .,"[ARGM-TMP: To the future] , [ARG0: the boys] [V: went] ."
23,The animals went to the party.,[ARG1: The animals] [V: went] [ARG4: to the party] .,"[ARG2: To the party] , [ARG1: the animals] [V: went] ."
29,The animals went to the party.,[ARG1: The animals] [V: went] [ARG4: to the party] .,"[ARG2: To the party] , [ARG1: the animals] [V: went] ."
43,The men went to the future.,[ARG0: The men] [V: went] [ARG4: to the future] .,"[ARGM-TMP: To the future] , [ARG0: the men] [V: went] ."
48,The girls went to the future.,[ARG0: The girls] [V: went] [ARG4: to the future] .,"[ARGM-TMP: To the future] , [ARG0: the girls] [V: went] ."


In [18]:
df_directive.to_csv("../results/inversion/irective_results_bilstm.csv")
df_locative.to_csv("../results/inversion/locative_results_bilstm.csv")

In [ ]:
print("error rate locative BiLSTM:")
print(failure_rate_locative_bilstm)
print("error rate directive BiLSTM:")
print(failure_rate_directive_bilstm)
print("error rate locative BERT:")
print(failure_rate_locative_bert)
print("error rate directive BERT:")
print(failure_rate_directive_bert)